En este archivo puedes escribir lo que estimes conveniente. Te recomendamos detallar tu solución y todas las suposiciones que estás considerando. Aquí puedes ejecutar las funciones que definiste en los otros archivos de la carpeta src, medir el tiempo, memoria, etc.

Se cargan las bibliotecas necesarias para el correcto funcionamiento de las funciones siguientes.

In [39]:
from typing import List, Tuple
from datetime import datetime
import json
from google.cloud import storage

Implementación para lectura del archivo json desde GCP. Con los parámetros: nombre del bucket: latam-1, nombre del archivo: farmers-protest-tweets-2021-2-4.json, id del proyecto GCP: latam-challenge-417813

In [40]:
def read_json_from_gcp(bucket_name, file_name, project_id):
    # Inicializa el cliente de almacenamiento con el ID del proyecto
    storage_client = storage.Client(project=project_id)
    
    # Obtiene el bucket
    bucket = storage_client.bucket(bucket_name)
    
    # Obtiene el blob (archivo) del bucket
    blob = bucket.blob(file_name)
    
    # Descarga el contenido del archivo JSON como bytes
    json_content_bytes = blob.download_as_string()
    
    # Decodifica cada línea del archivo JSON
    json_data = [json.loads(line) for line in json_content_bytes.decode('utf-8').splitlines() if line.strip()]
    
    return json_data


# Nombre del bucket y del archivo JSON en GCP
bucket_name = "latam-1"
file_name = "farmers-protest-tweets-2021-2-4.json"

# ID de tu proyecto de Google Cloud
project_id = "latam-challenge-417813"

# Lee el archivo JSON desde el bucket
json_content = read_json_from_gcp(bucket_name, file_name, project_id)

# Imprime el primer elemento para verificar la carga
print(json_content[:1])

[{'url': 'https://twitter.com/ArjunSinghPanam/status/1364506249291784198', 'date': '2021-02-24T09:23:35+00:00', 'content': 'The world progresses while the Indian police and Govt are still trying to take India back to the horrific past through its tyranny. \n\n@narendramodi @DelhiPolice Shame on you. \n\n#ModiDontSellFarmers \n#FarmersProtest \n#FreeNodeepKaur https://t.co/es3kn0IQAF', 'renderedContent': 'The world progresses while the Indian police and Govt are still trying to take India back to the horrific past through its tyranny. \n\n@narendramodi @DelhiPolice Shame on you. \n\n#ModiDontSellFarmers \n#FarmersProtest \n#FreeNodeepKaur twitter.com/ravisinghka/st…', 'id': 1364506249291784198, 'user': {'username': 'ArjunSinghPanam', 'displayname': 'Arjun Singh Panam', 'id': 45091142, 'description': 'Global Citizen, Actor, Director: Sky is the roof above my head, the world is the road I travel, love is my food & mother earth is my bed. Roy in @CosmosMovie', 'rawDescription': 'Global Cit

Se asigna a variable file_path la función read_json_from_gcp(bucket_name, file_name, project_id), para así leer el archivo json desde la nube.

In [41]:
file_path = read_json_from_gcp("latam-1", "farmers-protest-tweets-2021-2-4.json", "latam-challenge-417813")

Se realiza la función q1_memory, con enfoque en el uso de memoria utilizando el archivo JSON desde GCP.

In [42]:
def q1_memory(file_path: str) -> List[Tuple[datetime.date, str]]:
    # Lista para almacenar tuplas de fecha y usuario más activo
    date_top_user = []
    
    # Diccionario para contar tweets por fecha
    date_tweets = {}
    
        
    for tweet in file_path:
        if 'date' in tweet:
            tweet_date = datetime.strptime(tweet['date'], '%Y-%m-%dT%H:%M:%S%z').date()
            user = tweet['user']['username']
            
            # Actualizar el conteo de tweets por fecha
            date_tweets[tweet_date] = date_tweets.get(tweet_date, 0) + 1
            
            # Actualizar el usuario más activo para esta fecha
            if not date_top_user or date_tweets[tweet_date] > date_tweets[date_top_user[-1][0]]:
                date_top_user.append((tweet_date, user))
                date_top_user.sort(key=lambda x: x[0], reverse=True)
                date_top_user = date_top_user[:10]
                    
    return date_top_user

# Ejemplo de uso
result = q1_memory(file_path)
print(result)

[(datetime.date(2021, 2, 24), 'ArjunSinghPanam'), (datetime.date(2021, 2, 23), 'Avni_here'), (datetime.date(2021, 2, 20), 'JBBal75'), (datetime.date(2021, 2, 18), 'LakhzClick'), (datetime.date(2021, 2, 17), 'kisan_HRY'), (datetime.date(2021, 2, 13), 'annuthatte'), (datetime.date(2021, 2, 12), 'prabhatsinghdn')]


Implementación de la función q1_time, con un enfoque en la optimización del tiempo.

In [43]:
def q1_time(tweets: List[dict]) -> List[Tuple[datetime.date, str]]:
    # Diccionario para almacenar la cantidad de tweets por fecha
    date_tweets = {}
    # Diccionario para almacenar el usuario más activo por fecha
    date_top_user = {}

    # Procesar cada tweet en la lista
    for tweet in tweets:
        # Obtener la fecha del tweet
        tweet_date = datetime.strptime(tweet['date'], "%Y-%m-%dT%H:%M:%S%z").date()
        user = tweet['user']['username']
        
        # Contar la cantidad de tweets por fecha
        date_tweets[tweet_date] = date_tweets.get(tweet_date, 0) + 1
        
        # Actualizar el usuario más activo para esta fecha
        if tweet_date not in date_top_user:
            date_top_user[tweet_date] = user
        else:
            current_tweets = date_tweets[tweet_date]
            max_tweets = date_tweets.get(date_top_user[tweet_date], 0)
            if current_tweets > max_tweets:
                date_top_user[tweet_date] = user
    
    # Obtener las top 10 fechas con más tweets
    top_dates = sorted(date_tweets.items(), key=lambda x: x[1], reverse=True)[:10]
    
    # Obtener el usuario más activo para cada fecha
    result = [(date, date_top_user[date]) for date, _ in top_dates]
    
    return result


# Ejemplo de uso
result = q1_time(file_path)
print(result)

[(datetime.date(2021, 2, 12), 'SikhVibes'), (datetime.date(2021, 2, 13), 'Gurpreetd86'), (datetime.date(2021, 2, 17), 'MovimentoGhadar'), (datetime.date(2021, 2, 16), 'Anumanhas11'), (datetime.date(2021, 2, 14), 'khush18'), (datetime.date(2021, 2, 18), 'rebelpacifist'), (datetime.date(2021, 2, 15), 'DeepSin79880831'), (datetime.date(2021, 2, 20), 'IndiaToday'), (datetime.date(2021, 2, 23), 'SahibSi39465273'), (datetime.date(2021, 2, 19), 'A_W_M_B')]


Los resultados son diferentes, lo cual es correcto dada la naturaleza de optimización de tiempo y memoria que se buscaba para cada función:


· "q1_time" se enfoca en la optimización de tiempo al procesar eficientemente todos los tweets para determinar las fechas con más publicaciones y el usuario más activo para cada una de esas fechas. Esto se logra sin preocuparse por el consumo de memoria adicional para almacenar todos los usuarios y sus tweets.

· "q1_memory", por otro lado, se enfoca en la optimización de memoria al mantener una lista de tuplas que contienen solo las fechas y usuarios más activos hasta el momento, evitando así almacenar todos los tweets y usuarios en memoria. Esto reduce la cantidad de memoria necesaria, aunque a costa de un poco más de tiempo debido a la necesidad de verificar y actualizar la lista de usuarios más activos en cada paso.

Espero que esto aclare por qué los outputs son distintos entre ambas funciones y cómo cada una cumple con el objetivo de optimización específico. Si tienes más preguntas o necesitas más ayuda, no dudes en preguntar.